# Level Break Out

### Load the data

In [1]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

df = pd.read_csv("EURUSD_Candlestick_1_D_BID_05.05.2003-28.10.2023.csv")

df.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)


df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)

df['EMA'] = ta.ema(df.close, length=50)
df.tail()

df=df[0:]
df.reset_index(drop=True, inplace=True)

### Trend detection

In [2]:
EMAsignal = [0]*len(df)
backcandles = 10

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [3]:
df

,Gmt time,open,high,low,close,volume,EMA,EMASignal
0,05.05.2003 00:00:00.000,1.12161,1.13009,1.12014,1.12924,1.276006e+06,NaN,0
1,06.05.2003 00:00:00.000,1.12921,1.14506,1.12723,1.14234,1.281467e+06,NaN,0
2,07.05.2003 00:00:00.000,1.14218,1.14323,1.13265,1.13494,1.272597e+06,NaN,0
3,08.05.2003 00:00:00.000,1.13507,1.15077,1.13006,1.14820,1.266581e+06,NaN,0
4,09.05.2003 00:00:00.000,1.14831,1.15366,1.14346,1.14867,1.273339e+06,NaN,0
...,...,...,...,...,...,...,...,...
6405,24.10.2023 00:00:00.000,1.06699,1.06942,1.05830,1.05949,2.703091e+05,1.064827,0
6406,25.10.2023 00:00:00.000,1.05952,1.06067,1.05601,1.05661,2.475389e+05,1.064505,0
6407,26.10.2023 00:00:00.000,1.05661,1.05691,1.05221,1.05638,2.871348e+05,1.064186,0
6408,27.10.2023 00:00:00.000,1.05637,1.05970,1.05354,1.05605,2.156052e+05,1.063867,0


In [4]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [5]:
window=6
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [6]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [7]:
dfpl = df[4300:4600]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [8]:
def detect_structure(candle, backcandles, window):
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0
    
    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    levelbreak = 0
    zone_width = 0.01
    if len(lows)==3:
        support_condition = True
        mean_low = lows.mean()
        for low in lows:
            if abs(low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle].close)>zone_width*2:
            levelbreak = 1

    if len(highs)==3:
        resistance_condition = True
        mean_high = highs.mean()
        for high in highs:
            if abs(high-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle].close-mean_high)>zone_width*2:
            levelbreak = 2
    return levelbreak


In [9]:
#df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=15))
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=40, window=6), axis=1)


In [10]:
df[df['pattern_detected']!=0].head(20)

,Gmt time,open,high,low,close,volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
1507,27.02.2008 00:00:00.000,1.49948,1.51435,1.49774,1.51150,858801.3125,1.470108,2,0,NaN,2
2418,25.01.2011 00:00:00.000,1.36425,1.37038,1.35742,1.36846,214800.8750,1.333875,0,0,NaN,2
2419,26.01.2011 00:00:00.000,1.36845,1.37212,1.36449,1.37097,182469.0625,1.335330,2,0,NaN,2
3049,30.01.2013 00:00:00.000,1.34874,1.35870,1.34817,1.35682,175367.4844,1.322583,2,0,NaN,2
3050,31.01.2013 00:00:00.000,1.35680,1.36165,1.35408,1.36115,166226.0625,1.324095,2,0,NaN,2
3051,01.02.2013 00:00:00.000,1.36115,1.37108,1.35860,1.36391,204622.9844,1.325656,2,1,1.37208,2
3052,03.02.2013 00:00:00.000,1.36521,1.36571,1.36449,1.36449,4513.7002,1.327179,2,0,NaN,2
3054,05.02.2013 00:00:00.000,1.35103,1.35972,1.34577,1.35825,197343.0469,1.329297,2,0,NaN,2
3649,31.12.2014 00:00:00.000,1.21613,1.21697,1.20968,1.20976,73325.1016,1.238216,1,0,NaN,1
3757,06.05.2015 00:00:00.000,1.11794,1.13700,1.11750,1.13391,240872.4063,1.096869,0,0,NaN,2


In [11]:
data = df[:5000].copy()
def SIGNAL():
    return data.pattern_detected
data.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)
data

,Gmt time,Open,High,Low,Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
0,05.05.2003 00:00:00.000,1.12161,1.13009,1.12014,1.12924,1.276006e+06,NaN,0,0,NaN,0
1,06.05.2003 00:00:00.000,1.12921,1.14506,1.12723,1.14234,1.281467e+06,NaN,0,0,NaN,0
2,07.05.2003 00:00:00.000,1.14218,1.14323,1.13265,1.13494,1.272597e+06,NaN,0,0,NaN,0
3,08.05.2003 00:00:00.000,1.13507,1.15077,1.13006,1.14820,1.266581e+06,NaN,0,0,NaN,0
4,09.05.2003 00:00:00.000,1.14831,1.15366,1.14346,1.14867,1.273339e+06,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
4995,22.04.2019 00:00:00.000,1.12440,1.12620,1.12357,1.12584,1.848423e+05,1.128744,0,0,NaN,0
4996,23.04.2019 00:00:00.000,1.12585,1.12612,1.11921,1.12237,3.667941e+05,1.128494,0,0,NaN,0
4997,24.04.2019 00:00:00.000,1.12239,1.12241,1.11406,1.11533,3.960929e+05,1.127978,0,0,NaN,0
4998,25.04.2019 00:00:00.000,1.11534,1.11623,1.11178,1.11353,3.468432e+05,1.127412,0,0,NaN,0


In [12]:
data['RSI'] = ta.rsi(data['Close'])
data.set_index("Gmt time", inplace=True)
data.index = pd.to_datetime(data.index, format='%d.%m.%Y %H:%M:%S.%f').floor('S')
data

,Open,High,Low,Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected,RSI
Gmt time,,,,,,,,,,,
2003-05-05,1.12161,1.13009,1.12014,1.12924,1.276006e+06,NaN,0,0,NaN,0,NaN
2003-05-06,1.12921,1.14506,1.12723,1.14234,1.281467e+06,NaN,0,0,NaN,0,NaN
2003-05-07,1.14218,1.14323,1.13265,1.13494,1.272597e+06,NaN,0,0,NaN,0,NaN
2003-05-08,1.13507,1.15077,1.13006,1.14820,1.266581e+06,NaN,0,0,NaN,0,NaN
2003-05-09,1.14831,1.15366,1.14346,1.14867,1.273339e+06,NaN,0,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2019-04-22,1.12440,1.12620,1.12357,1.12584,1.848423e+05,1.128744,0,0,NaN,0,47.863943
2019-04-23,1.12585,1.12612,1.11921,1.12237,3.667941e+05,1.128494,0,0,NaN,0,42.087795
2019-04-24,1.12239,1.12241,1.11406,1.11533,3.960929e+05,1.127978,0,0,NaN,0,33.306049


In [13]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 10000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.03
        
        #Close trades if RSI is above 70 for long positions and below 30 for short positions
        for trade in self.trades:
            if trade.is_long and self.data.RSI[-1] > 80:
                trade.close()
            elif trade.is_short and self.data.RSI[-1] < 20:
                trade.close()

        if self.signal!=0 and len(self.trades)==0 and self.data.pattern_detected==2:
            sl = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]+sldiff*TPSLRatio
            self.buy(sl=sl, tp=tp, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.pattern_detected==1:         
            sl = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]-sldiff*TPSLRatio
            self.sell(sl=sl, tp=tp, size=self.mysize)

bt = Backtest(data, MyStrat, cash=10000, margin=1/5)
stat = bt.run()
stat

Start                     2003-05-05 00:00:00
End                       2019-04-26 00:00:00
Duration                   5835 days 00:00:00
Exposure Time [%]                        3.64
Equity Final [$]                    10951.735
Equity Peak [$]                     11165.976
Return [%]                            9.51735
Buy & Hold Return [%]               -1.323899
Return (Ann.) [%]                    0.459252
Volatility (Ann.) [%]                1.875674
Sharpe Ratio                         0.244846
Sortino Ratio                        0.358288
Calmar Ratio                         0.053188
Max. Drawdown [%]                   -8.634543
Avg. Drawdown [%]                   -1.604492
Max. Drawdown Duration     2273 days 00:00:00
Avg. Drawdown Duration      196 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                       6.140175
Worst Trade [%]                     -3.001675
Avg. Trade [%]                    

In [14]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 5000 #5000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.03

        if len(self.trades)==1:
            for trade in self.trades:
                trade.sl = trade.entry_price
            #self.trades[-1].sl = self.trades[-1].entry_price

        # Close trades if RSI is above 70 for long positions and below 30 for short positions
        # for trade in self.trades:
        #     if trade.is_long and self.data.RSI[-1] > 80:
        #         trade.close()
        #     elif trade.is_short and self.data.RSI[-1] < 20:
        #         trade.close()
            
        if self.signal!=0 and len(self.trades)==0 and self.data.pattern_detected==2:
            sl1 = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]+sldiff*TPSLRatio
            tp2 = self.data.Close[-1]+sldiff
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.pattern_detected==1:         
            sl1 = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]-sldiff*TPSLRatio
            tp2 = self.data.Close[-1]-sldiff
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(data, MyStrat, cash=10000, margin=1/5)
stat = bt.run()
stat

Start                     2003-05-05 00:00:00
End                       2019-04-26 00:00:00
Duration                   5835 days 00:00:00
Exposure Time [%]                         6.1
Equity Final [$]                   11454.3995
Equity Peak [$]                     11734.092
Return [%]                          14.543995
Buy & Hold Return [%]               -1.323899
Return (Ann.) [%]                    0.686723
Volatility (Ann.) [%]                2.100747
Sharpe Ratio                         0.326895
Sortino Ratio                        0.496492
Calmar Ratio                         0.093537
Max. Drawdown [%]                   -7.341727
Avg. Drawdown [%]                   -1.311998
Max. Drawdown Duration     1442 days 00:00:00
Avg. Drawdown Duration      103 days 00:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                       6.140175
Worst Trade [%]                     -3.001675
Avg. Trade [%]                    

In [15]:
bt.plot()

Row(id='1475', ...)